In [1]:
import pandas as pd
dfSite = pd.read_excel('VisitorEntrancesV3.xlsx')
dfTemperature = pd.read_csv('Temperature-2016-2020-Byte-Dagan.csv')

גלי חום – אנא הגדירו יום שנחשב כגל חום כיום שנמצא ברצף של לפחות שלושה ימים בהם טמפ' המקסימום היא לפחות 33 מעלות בבית דגן (זה אמור להיות מספיק כוללני)

אנא כתבו כמה ימים היו בגלי חום מתוך כל הימים וכמה ימים היו בחריגה במזהמי האוויר (הארצית)

טמפרטורה – אנא ודאו שמדובר בטמפ' מקסימום יומית


In [16]:
dfSite.head(1)

,Date,Site_Name,Israelis_Count,region_Central,region_Judea_Samaria,region_North,region_South,is_weekend,is_camping,special_activity,...,Season_autumn,Season_spring,Season_summer,Season_winter,last_year_visitors,Exceeded_pm10_in_Israel,Site_Exceeded,IsHoliday,IsVacation,IsHeritage
0,2016-03-02,Akhziv,211,0,0,1,0,0,1,0,...,0,1,0,0,NaN,0,0,False,False,False


In [17]:
dfTemperature.head(1)

,Date,Temperature
0,1/1/2016,13.9


In [18]:
#Check what type Date is.
dfTemperature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         1461 non-null   object 
 1   Temperature  1461 non-null   float64
dtypes: float64(1), object(1)
memory usage: 23.0+ KB


In [19]:
#Convert Date type from obj to datetime type
dfTemperature.Date = pd.to_datetime(dfTemperature.Date,dayfirst=True)


In [20]:
#check change
dfTemperature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         1461 non-null   datetime64[ns]
 1   Temperature  1461 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 23.0 KB


In [26]:
#update Temperature in dfSite
#merge with dfTemperature.
dfMerge = dfSite.drop('Temperature',axis=1).merge(dfTemperature, on='Date',how='left')
dfMerge.isna().sum()
dfMerge

,Date,Site_Name,Israelis_Count,region_Central,region_Judea_Samaria,region_North,region_South,is_weekend,is_camping,special_activity,...,Season_spring,Season_summer,Season_winter,last_year_visitors,Exceeded_pm10_in_Israel,Site_Exceeded,IsHoliday,IsVacation,IsHeritage,Temperature
0,2016-03-02,Akhziv,211,0,0,1,0,0,1,0,...,1,0,0,NaN,0,0,False,False,False,30.1
1,2016-03-17,Akhziv,186,0,0,1,0,0,1,0,...,1,0,0,NaN,0,0,False,False,False,19.2
2,2016-03-21,Akhziv,59,0,0,1,0,0,1,0,...,1,0,0,NaN,0,0,False,False,False,20.1
3,2016-03-25,Akhziv,6,0,0,1,0,1,1,0,...,1,0,0,NaN,1,0,True,True,False,26.5
4,2016-04-01,Akhziv,9,0,0,1,0,1,1,0,...,1,0,0,NaN,0,0,False,True,False,27.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75160,2019-12-23,Yehudiya,8,0,0,1,0,0,0,0,...,0,0,1,2.0,0,1,True,False,False,20.8
75161,2019-12-24,Yehudiya,15,0,0,1,0,0,0,0,...,0,0,1,NaN,0,0,True,True,False,22.2
75162,2019-12-27,Yehudiya,1,0,0,1,0,1,0,0,...,0,0,1,NaN,0,0,True,True,False,15.2
75163,2019-12-28,Yehudiya,4,0,0,1,0,1,0,0,...,0,0,1,NaN,0,0,True,True,False,12.8


לאחר עדכון הטמפ' נעדכן עכשיו את הגל חום

In [45]:
dfMerge = dfMerge.drop('is_HeatWave',axis=1)

In [5]:
dfMerge['isHeatwave'] = 0
import function
dfMerge.sort_values(['Site_Name','Date'],inplace=True)
dfMerge.reset_index(drop=True,inplace=True)

for i in range(2,len(dfMerge)):
    index0 = i-2
    index1 = i-1
    index2 = i
    date0,date2 = dfMerge.loc[index0,'Date'],dfMerge.loc[index2,'Date']
    #check if dates are continuous
    if not function.date_diff(date0,date2,2):
        continue
    
    isfirstDayUnder33 = dfMerge.loc[index0,'Temperature']<32.2
    isSecDayUnder33 = dfMerge.loc[index1,'Temperature']<32.2
    isThrDayUnder33 = dfMerge.loc[index2,'Temperature']<32.2
    
    #check if one of the days are not over 32.2
    if isfirstDayUnder33 | isSecDayUnder33 | isThrDayUnder33:
        continue
    
    dfMerge.loc[index0,'isHeatwave'] = 1
    dfMerge.loc[index1,'isHeatwave'] = 1
    dfMerge.loc[index2,'isHeatwave'] = 1


In [7]:
dfMerge[['Date','Site_Name','Temperature','isHeatwave']][dfMerge['isHeatwave']==1]

,Date,Site_Name,Temperature,isHeatwave
45,2016-05-14,Akhziv,37.5,1
46,2016-05-15,Akhziv,40.6,1
47,2016-05-16,Akhziv,38.2,1
65,2016-06-03,Akhziv,36.6,1
66,2016-06-04,Akhziv,40.2,1
...,...,...,...,...
75055,2019-08-29,Yehudiya,32.6,1
75056,2019-08-30,Yehudiya,32.5,1
75057,2019-08-31,Yehudiya,32.7,1
75058,2019-09-01,Yehudiya,32.4,1


In [8]:
dfMerge.to_excel('VisitorEntrancesV3.xlsx',index=False)